In [2]:
#make sure you have these versions of TensorFlow and Keras
# !python -m pip install tensorflow

In [3]:
import tensorflow as tf 
print(tf.__version__)

2.16.1


In [4]:
import keras
keras.__version__

'3.3.3'

In [5]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
train=pd.read_table('train.txt', delimiter = ';', header=None, )
val=pd.read_table('val.txt', delimiter = ';', header=None, )
test=pd.read_table('test.txt', delimiter = ';', header=None, )

data = pd.concat([train ,  val , test])
data.columns = ["text", "label"]



In [6]:
data

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [7]:
data.isna().any(axis=1).sum()

0

In [8]:
#text preprocessing
ps = PorterStemmer()

def preprocess(line):
    review = re.sub('[^a-zA-Z]', ' ', line) #leave only characters from a to z
    review = review.lower() #lower the text
    review = review.split() #turn string into list of words
    #apply Stemming 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #delete stop words like I, and ,OR   review = ' '.join(review)
    #trun list into sentences
    return " ".join(review)


In [9]:
data['text']=data['text'].apply(lambda x: preprocess(x))


In [10]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
data['N_label'] = label_encoder.fit_transform(data['label'])


In [11]:
data['text']

0                                       didnt feel humili
1       go feel hopeless damn hope around someon care ...
2                    im grab minut post feel greedi wrong
3          ever feel nostalg fireplac know still properti
4                                            feel grouchi
                              ...                        
1995    keep feel like someon unkind wrong think get b...
1996              im feel littl cranki neg doctor appoint
1997                feel use peopl give great feel achiev
1998    im feel comfort derbi feel though start step s...
1999    feel weird meet w peopl text like dont talk fa...
Name: text, Length: 20000, dtype: object

In [12]:
# Creating the Bag of Words model by applying Countvectorizer -convert textual data to numerical data


from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000,ngram_range=(1,3))#example: the course was long-> [the,the course,the course was,course, course was, course was long,...]

data_cv = cv.fit_transform(data['text']).toarray()




In [13]:
data_cv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
#X_train, X_test, y_train, y_test=data_cv,test_cv,train['N_label'],test['N_label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(data_cv, data['N_label'], test_size=0.25, random_state=42)


In [15]:
# first neural network with keras tutorial
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(6, activation='softmax'))
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

/Users/leonzhang/miniconda3/envs/sch_env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 530us/step - accuracy: 0.4650 - loss: 1.3734
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 475us/step - accuracy: 0.9126 - loss: 0.2847
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 453us/step - accuracy: 0.9529 - loss: 0.1523
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 458us/step - accuracy: 0.9678 - loss: 0.0977
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 465us/step - accuracy: 0.9807 - loss: 0.0669
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 458us/step - accuracy: 0.9860 - loss: 0.0482
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 461us/step - accuracy: 0.9882 - loss: 0.0367
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - accuracy: 0.9921 - loss: 0.0272
Epoch 9/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 457us/step - accuracy: 0.9934 - loss: 0.0225
Epoch 10/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 467us/step - accuracy: 0.9940 - loss: 0.0198
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.9956 - loss: 0.0148
Accuracy: 99.51


In [16]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - accuracy: 0.8551 - loss: 0.8165
Accuracy: 84.78


In [17]:
import numpy as np
text="I do not feel good"
text=preprocess(text)
array = cv.transform([text]).toarray()
pred = model.predict(array)
a=np.argmax(pred, axis=1)
label_encoder.inverse_transform(a)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


'joy'

In [371]:
tf.keras.models.save_model(model,'my_model.h5')


In [370]:
import pickle
pickle.dump(label_encoder, open('encoder.pkl', 'wb'))
pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))
